# Packages

In [119]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import tifffile as tiff
import os
from tqdm import tqdm
import pandas as pd
import pickle
import time

import cv2

# Utility Functions

In [132]:
def mask_rle_to_2d(rle_mask, dx, dy):
    """
    converts mask from run length encoding to 2D numpy array
    """

    mask = np.zeros(dx*dy, dtype=np.uint8)
    s = rle_mask.split()  # split the rle encoding

    for i in range(len(s)//2):
        start = int(s[2*i])-1
        length = int(s[2*i+1])
        mask[start:start+length] = 1

    mask = mask.reshape(dy, dx).T

    return mask
        
    
def mask_2d_to_rle(mask_2d):
    """
    Takes a 2D mask of 0/1 and returns the run length encoded form

    """
    mask = mask_2d.T.reshape(-1) #order by columns and flatten to 1D
    mask_padded = np.pad(mask, 1) #pad zero on both sides
    #find the start positions of the 1's
    starts = np.where(mask_padded[:-1] == 1 & (mask_padded[1:] == 0))[0]
    #find the end positions of 1's for each run
    ends = np.where((mask_padded[:-1] == 0) & (mask_padded[1:] == 1))[0]
    
    rle = np.zeros(2*len(starts))
    print(starts.shape, ends.shape, rle.shape)
    rle[::2] = starts
    #length of each run = end position - start position
    rle[1::2] = ends - starts

    return rle

In [3]:
def get_padsize(img, reduce, sz):

    shape = img.shape
    print(shape)

    pad0 = (reduce*sz - shape[0] % (reduce*sz)) % (reduce*sz)
    pad1 = (reduce*sz - shape[1] % (reduce*sz)) % (reduce*sz)
    pad_x = (pad0//2, pad0-pad0//2)
    pad_y = (pad1//2, pad1-pad1//2)

    return pad_x, pad_y


def check_threshold(img_BGR, sat_threshold, pixcount_th):

    """
    checks if an input image passes the threshold conditions:
    conditions:
    not black--> sum of pixels exceeed a threshold = pixcount_th
    saturation --> number of pixels with saturation > sat_threshold exceeds pixcount_th
    Returns:
    True if both conditions are met else False
    """
    #if most of the pixels are black, return False
    #edge of each image is typically black
    if img_BGR.sum() < pixcount_th:
        return False

    #convert to hue, saturation, Value in openCV
    hsv = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    # if less than prefined number of values are above a saturation threshold, return False
    #this is typically the gray background around the biological object
    if (s > sat_threshold).sum() < pixcount_th:
        return False

    return True

In [111]:
class Image():
    def __init__(self, img, img_name =None, sat_threshold=40, pixcount_th=200):
        self.img = img
        self.shape = img.shape
        self.name = img_name

        self.image_reshape()
        self.dx = self.shape[0]
        self.dy = self.shape[1]

        self.tile_size = None

        self.pad_x = None
        self.pad_y = None
        self.tiled_img = None

        self.mask_rle = None
        self.mask_2d = None
        self.tiled_mask = None
        
    
    def image_reshape(self):
        
        if len(self.shape) == 5:
            self.img = np.transpose(self.img.squeeze(), (1, 2, 0))
            self.shape = self.img.shape
            
    
    def split_image_mask_into_tiles(self, reduce=1, sz=512):
     
        self.tile_size = sz

        self.pad_x, self.pad_y = get_padsize(self.img, reduce, sz)
        print(self.pad_x, self.pad_y)
        #Create padded Image and padded mask2D
        img_padded  = np.pad(self.img, [self.pad_x, self.pad_y, (0, 0)], constant_values=0)
        
        print("shape of image after padding:: ", img_padded.shape,
            img_padded.shape[0]//sz, img_padded.shape[1]//sz)

        #tile the padded image
        img_reshaped = img_padded.reshape(
            img_padded.shape[0]//sz, sz, img_padded.shape[1]//sz, sz, 3)
        img_reshaped = img_reshaped.transpose(0, 2, 1, 3, 4).reshape(-1, sz, sz, 3)
        self.tiled_img = img_reshaped

        '''
        mask_padded = np.pad(self.mask_2d, [self.pad_x, self.pad_y], constant_values = 0)
        print("shape of mask after padding:: ", mask_padded.shape,
              mask_padded.shape[0]//sz, mask_padded.shape[1]//sz)
        #tile the padded mask2D
        mask_reshaped = mask_padded.reshape(
            mask_padded.shape[0]//sz, sz, mask_padded.shape[1]//sz, sz)
        mask_reshaped = mask_reshaped.transpose(
            0, 2, 1, 3).reshape(-1, sz, sz)

        self.tiled_mask = mask_reshaped
        ''' 
            
        
    def reconstruct_original_from_padded_tiled(self, tiled_padded_mask):
        n = tiled_padded_mask.shape[0] #number of tiles after padding
        tile_size = self.tile_size
        (pad_x_l, pad_x_r) = self.pad_x
        (pad_y_l, pad_y_r) = self.pad_y

        dx_padded = self.dx + pad_x_l + pad_x_r
        dy_padded = self.dy + pad_y_l + pad_y_r

        n_x = dx_padded //tile_size
        n_y = dy_padded//tile_size

        assert (n == n_x*n_y), "dimensions don't match"

        mask_untiled = tiled_padded_mask.reshape(n_x, n_y, tile_size, tile_size)
        mask_untiled = mask_untiled.transpose(0,2,1,3)
        mask_padded = mask_untiled.reshape(n_x*tile_size, n_y*tile_size)

        mask_unpadded = mask_padded[pad_x_l: - pad_x_r, pad_y_l: -pad_y_r]

        assert (self.dx == mask_unpadded.shape[0]), \
            "shape of original image doesn't match with unpadded mask along dim = 0"
        assert (self.dy == mask_unpadded.shape[1]), \
            "shape of original image doesn't match with unpadded mask along dim = 1"

        return mask_unpadded

# Model

In [38]:
class BaseModel(nn.Module):
    
    def __init__(self):
        super(BaseModel, self).__init__()

    def forward(self):
        raise NotImplementedError

    def __str__(self):
        model_params = filter(lambda x: x.requires_grad, self.parameters())

        return super(BaseModel, self).__str__()
    
    
class Conv2x(nn.Module):
    '''
    preserves the the size of the image
    '''
    def __init__(self, in_ch, out_ch, inner_ch=None):
        super(Conv2x, self).__init__()
        self.in_ch = in_ch
        self.out_ch = out_ch
        self.inner_ch = out_ch//2 if inner_ch is None else inner_ch

        self.conv2d_1 = nn.Conv2d(self.in_ch, self.inner_ch,
                                  kernel_size=3, padding=1, bias=False)
        self.conv2d_2 = nn.Conv2d(self.inner_ch, self.out_ch,
                                  kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inner_ch)
        self.bn2 = nn.BatchNorm2d(self.out_ch)

    def forward(self, x):
        x = self.conv2d_1(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2d_2(x)
        x = self.bn2(x)
        x = F.relu(x)

        return x
    
    
class encoder(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(encoder, self).__init__()
        self.conv2x = Conv2x(in_ch, out_ch)
        self.pool = nn.MaxPool2d(kernel_size=2, ceil_mode=True)

    def forward(self, x):
        x = self.conv2x(x)
        x = self.pool(x)
        return x
    
    
class decoder(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(decoder, self).__init__()
        self.transposeconv = nn.ConvTranspose2d(
            in_ch, in_ch//2, kernel_size=2, stride=2)
        self.conv2x = Conv2x(in_ch, out_ch)


    def forward(self, x_down, x_up, interpolate=True):

        x_up = self.transposeconv(x_up)

        #check for matching dims before concatenating

        if (x_up.size(2) != x_up.size(2)) or (x_up.size(3) != x_up.size(3)):
            if interpolate:
                x_up = F.interpolate(x_up, size=(x_down.size(2), x_down.size(3)),
                mode="bilinear", align_corners=True)
        
        #Concat features from down conv channel and current up-conv
        #along channel dim =1
        x_up = torch.cat([x_up, x_down], dim=1) 
        x_up = self.conv2x(x_up)

        return x_up
    


class UNet(BaseModel):

    def __init__(self, in_ch=3, conv_channels=[16, 32, 64, 128, 256]):
        super(UNet, self).__init__()

        self.conv_channels = conv_channels
        self.conv_start = Conv2x(in_ch, conv_channels[0]) #output_size = input_size
        self.down1 = encoder(conv_channels[0], conv_channels[1])   #output_size = input_size/2
        self.down2 = encoder(conv_channels[1], conv_channels[2])   #output_size = input_size/2
        self.down3 = encoder(conv_channels[2], conv_channels[3])   #output_size = input_size/2
        self.down4 = encoder(conv_channels[3], conv_channels[4])   #output_size = input_size/2

        self.conv_middle = Conv2x(conv_channels[4], conv_channels[4]) #output_size = input_size

        self.up4 = decoder(conv_channels[4], conv_channels[3]) #output_size = input_size * 2
        self.up3 = decoder(conv_channels[3], conv_channels[2]) #output_size = input_size * 2
        self.up2 = decoder(conv_channels[2], conv_channels[1]) #output_size = input_size * 2
        self.up1 = decoder(conv_channels[1], conv_channels[0]) #output_size = input_size * 2

        self.final_conv = nn.Conv2d(self.conv_channels[0], 1, kernel_size=1)

        self.init_params()
    
    
    def init_params(self):
        for module in self.modules():
            if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
                nn.init.kaiming_normal_(module.weight)
                if module.bias is not None:
                    module.bias.data.zero_()

            elif isinstance(module, nn.BatchNorm2d):
                module.weight.data.fill_(1)
                module.bias.data.zero_()
                
    
    
    def forward(self, x):
        # size of x = [B, _, nx, ny]
        
        x1 = self.conv_start(x)  # size of x = [B, self.conv_channels[0], nx, ny]
        x2 = self.down1(x1)  # size of x = [B, self.conv_channels[1], nx/2, ny/2]
        x3 = self.down2(x2)  # size of x = [B, self.conv_channels[2], nx/4, ny/4]
        x4 = self.down3(x3)  # size of x = [B, self.conv_channels[3], nx/8, ny/8]
        x5 = self.down4(x4)  # size of x = [B, self.conv_channels[4], nx/16, ny/16]

        x = self.conv_middle(x5)  # size of x = [B, self.conv_channels[4], nx/16, ny/16]

        x = self.up4(x4, x)       # size of x = [B, self.conv_channels[3], nx/8, ny/8]
        x = self.up3(x3, x)       # size of x = [B, self.conv_channels[2], nx/4, ny/4]
        x = self.up2(x2, x)       # size of x = [B, self.conv_channels[1], nx/2, ny/2]
        x = self.up1(x1, x)       # size of x = [B, self.conv_channels[0], nx, ny]

        x = self.final_conv(x)

        return x

# Metric 

In [148]:
def metric_dice_iou(output, target, smooth = 0.005):
        tp = (output * target).sum() #intersection
        fp = (output * (1.0 - target)).sum() #false positives
        fn = ((1.0 - output) * target).sum() #false negatives
        dice = np.mean((2.0 * tp + smooth) / (2 * tp + fp + fn + smooth))
        iou = np.mean((tp + smooth) / (tp + fp + fn + smooth))
        print(tp, fp, fn)
        print(output.sum(), target.sum())

        return dice, iou

### Dataset for a single image with batch size =1

In [96]:
class Dataset_Image(Dataset):

    def __init__(self, img_dir, img_file, mean, std_dev, tile_size, sat_threshold=40, pixcount_th=200, transform=None):
        super(Dataset_Image, self).__init__()
        
        self.sat_threshold = sat_threshold
        self.pixcount_th = pixcount_th
        self.transform = transform        
        self.normalize = transforms.Normalize(mean, std_dev)
        
        #read image from disk 
        print("reading image from disk")
        img_raw = tiff.imread(os.path.join(img_dir, img_file))
        raw_file_name = img_file.split('.')[0]
    
        self.raw_img = Image(img_raw,img_name=raw_file_name, sat_threshold=sat_threshold, pixcount_th=pixcount_th)
        #call Image.split_image_mask_into_tiles(self, reduce=1, sz=512) to create
        print("tiling image")
        self.raw_img.split_image_mask_into_tiles(reduce=1, sz=tile_size)

        self.tiled_img = self.raw_img.tiled_img
        
        self.len = self.__len__()


   
    def __len__(self):
        return len(self.tiled_img)

    
    def __getitem__(self, idx):
        img = self.tiled_img[idx]
        thresold_pass = check_threshold(img, self.sat_threshold, self.pixcount_th)
        img = self.normalize(transforms.ToTensor()(img))
        
        return img, thresold_pass




# Perform inference

In [156]:
def inference_single_image(img_dir, img_file=None, tile_size = 512, user=1):
    '''
    Function for inference on a single image
    '''
    sat_threshold=40
    pixcount_th=200
    
    #instantiate model
    model = UNet() 
    # load state dict from saved model dict
    print("loading saved model")
    model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict'])

    model.eval()
    #model._reset_metrics()
    #tbar = tqdm(self.val_loader, ncols=100)
    
    mean = [0.68912, 0.47454, 0.6486]
    std_dev = [0.13275, 0.23647, 0.15536]
    
    sat_threshold=40
    pixcount_th=200
    
    inf_dataset = Dataset_Image(datadir_train, img_file, mean, std_dev, tile_size, sat_threshold=sat_threshold, pixcount_th=pixcount_th)
    inf_dataloader = DataLoader(inf_dataset, batch_size=1)

    with torch.no_grad(): 
        tile_count = inf_dataset.len

        total_tiles = 0
        tiles_idx = []
        print(tile_count)
        mask_pred_tiled = np.empty((tile_count,tile_size,tile_size))
        
        #loop over the tiles
        for i, (img_tile, threshold_pass) in enumerate(inf_dataloader):
            
            #img_BGR = tiled_img[i, :, :, :]
            
            if threshold_pass:
                #model_input = torch.unsqueeze(torch.tensor(img_tile),0)
                print("mask prediction :: {}".format(i))
                out = torch.squeeze(model(img_tile))
                out = (out > 0).numpy().astype(np.uint8)
                #print("shape of model output:: ", out.shape)
                        
            else:
                #print("no inference needed for tile :: {}".format(i))
                out = np.zeros((tile_size, tile_size))

            #accumulate the predictions into a big np array
            mask_pred_tiled[i] = out
        #prediction size = [total_tiles, sz, sz]

        
    return mask_pred_tiled, inf_dataset.raw_img



In [157]:
#specify datadir
user = 1
if user == 0:
    masks = '/Users/Ethan/Documents/Documents/Documents - Ethan’s MacBook Pro/python/kidney_files/hubmap-kidney-segmentation/train.csv'
    checkpoint_path = '/Users/Ethan/Documents/Documents/Documents - Ethan’s MacBook Pro/python/kidney_files/hubmap-kidney-segmentation/model_checkpoints.pth'
    datadir_train = '/Users/Ethan/Documents/Documents/Documents - Ethan’s MacBook Pro/python/kidney_files/hubmap-kidney-segmentation/train'

else:  
    masks = '/media/bony/Ganga_HDD_3TB/Ganges_Backup/Machine_Learning/HuBMAP_Hacking_Kidney/hubmap-kidney-segmentation/train.csv/'
    checkpoint_path = '/media/bony/Ganga_HDD_3TB/Ganges_Backup/Machine_Learning/HuBMAP_Hacking_Kidney/hubmap-kidney-segmentation/code/saved_checkpoints/UNet_18_01_08_00_30.pth'
    datadir_train = '/media/bony/Ganga_HDD_3TB/Ganges_Backup/Machine_Learning/HuBMAP_Hacking_Kidney/hubmap-kidney-segmentation/train/'

    
#read an image  
img_index = 1
raw_image_files = [f for f in os.listdir(datadir_train) if "tiff" in f]
img_file = raw_image_files[img_index]


#call this function for all the prediction images
t0 = time.time()

mask_pred_rle, raw_img_class = inference_single_image(datadir_train, img_file, tile_size=512)
print(time.time()-t0)

#recover predicted mask with size matching image size
mask_pred_2d = raw_img_class.reconstruct_original_from_padded_tiled(mask_pred_rle)

#test on images where mask label is available
masks_train = '/media/bony/Ganga_HDD_3TB/Ganges_Backup/Machine_Learning/HuBMAP_Hacking_Kidney/hubmap-kidney-segmentation/train.csv'
mask_rle_df = pd.read_csv(masks_train).set_index('id')

img_file_name = img_file.split('.')[0]
mask_rle = mask_rle_df.loc[img_file_name, 'encoding']
dx = raw_img_class.dx
dy = raw_img_class.dy
mask_label_2d = mask_rle_to_2d(mask_rle, dx, dy)

#check metrics 
metric_dice_iou(mask_pred_2d, mask_label_2d, smooth = 0.005)


loading saved model
reading image from disk
tiling image
(26780, 32220, 3)
(178, 178) (18, 18)
shape of image after padding::  (27136, 32256, 3) 53 63
3339
mask prediction :: 36
mask prediction :: 37
mask prediction :: 38
mask prediction :: 39
mask prediction :: 40
mask prediction :: 41
mask prediction :: 42
mask prediction :: 43
mask prediction :: 44
mask prediction :: 45
mask prediction :: 46
mask prediction :: 48
mask prediction :: 49
mask prediction :: 51
mask prediction :: 52
mask prediction :: 53
mask prediction :: 76
mask prediction :: 77
mask prediction :: 78
mask prediction :: 79
mask prediction :: 80
mask prediction :: 81
mask prediction :: 82
mask prediction :: 90
mask prediction :: 95
mask prediction :: 96
mask prediction :: 97
mask prediction :: 98
mask prediction :: 99
mask prediction :: 100
mask prediction :: 101
mask prediction :: 102
mask prediction :: 103
mask prediction :: 104
mask prediction :: 105
mask prediction :: 106
mask prediction :: 107
mask prediction :: 108

mask prediction :: 543
mask prediction :: 544
mask prediction :: 545
mask prediction :: 546
mask prediction :: 547
mask prediction :: 548
mask prediction :: 549
mask prediction :: 550
mask prediction :: 551
mask prediction :: 552
mask prediction :: 553
mask prediction :: 554
mask prediction :: 555
mask prediction :: 556
mask prediction :: 581
mask prediction :: 582
mask prediction :: 583
mask prediction :: 584
mask prediction :: 585
mask prediction :: 586
mask prediction :: 587
mask prediction :: 588
mask prediction :: 589
mask prediction :: 590
mask prediction :: 591
mask prediction :: 592
mask prediction :: 593
mask prediction :: 594
mask prediction :: 595
mask prediction :: 596
mask prediction :: 597
mask prediction :: 598
mask prediction :: 599
mask prediction :: 600
mask prediction :: 601
mask prediction :: 602
mask prediction :: 603
mask prediction :: 604
mask prediction :: 605
mask prediction :: 606
mask prediction :: 607
mask prediction :: 608
mask prediction :: 609
mask predic

mask prediction :: 1083
mask prediction :: 1084
mask prediction :: 1085
mask prediction :: 1086
mask prediction :: 1087
mask prediction :: 1088
mask prediction :: 1089
mask prediction :: 1090
mask prediction :: 1091
mask prediction :: 1092
mask prediction :: 1093
mask prediction :: 1094
mask prediction :: 1095
mask prediction :: 1096
mask prediction :: 1098
mask prediction :: 1099
mask prediction :: 1100
mask prediction :: 1101
mask prediction :: 1102
mask prediction :: 1103
mask prediction :: 1104
mask prediction :: 1105
mask prediction :: 1106
mask prediction :: 1107
mask prediction :: 1108
mask prediction :: 1109
mask prediction :: 1110
mask prediction :: 1111
mask prediction :: 1112
mask prediction :: 1113
mask prediction :: 1114
mask prediction :: 1115
mask prediction :: 1116
mask prediction :: 1117
mask prediction :: 1118
mask prediction :: 1119
mask prediction :: 1120
mask prediction :: 1121
mask prediction :: 1122
mask prediction :: 1123
mask prediction :: 1124
mask prediction 

mask prediction :: 1485
mask prediction :: 1486
mask prediction :: 1487
mask prediction :: 1488
mask prediction :: 1489
mask prediction :: 1490
mask prediction :: 1491
mask prediction :: 1492
mask prediction :: 1493
mask prediction :: 1494
mask prediction :: 1495
mask prediction :: 1496
mask prediction :: 1497
mask prediction :: 1498
mask prediction :: 1499
mask prediction :: 1500
mask prediction :: 1501
mask prediction :: 1502
mask prediction :: 1503
mask prediction :: 1504
mask prediction :: 1505
mask prediction :: 1506
mask prediction :: 1507
mask prediction :: 1508
mask prediction :: 1509
mask prediction :: 1517
mask prediction :: 1518
mask prediction :: 1519
mask prediction :: 1520
mask prediction :: 1521
mask prediction :: 1522
mask prediction :: 1523
mask prediction :: 1524
mask prediction :: 1525
mask prediction :: 1526
mask prediction :: 1527
mask prediction :: 1528
mask prediction :: 1529
mask prediction :: 1530
mask prediction :: 1531
mask prediction :: 1532
mask prediction 

mask prediction :: 1864
mask prediction :: 1865
mask prediction :: 1866
mask prediction :: 1867
mask prediction :: 1868
mask prediction :: 1869
mask prediction :: 1870
mask prediction :: 1871
mask prediction :: 1872
mask prediction :: 1873
mask prediction :: 1874
mask prediction :: 1875
mask prediction :: 1876
mask prediction :: 1877
mask prediction :: 1878
mask prediction :: 1879
mask prediction :: 1880
mask prediction :: 1881
mask prediction :: 1882
mask prediction :: 1883
mask prediction :: 1884
mask prediction :: 1885
mask prediction :: 1886
mask prediction :: 1887
mask prediction :: 1888
mask prediction :: 1894
mask prediction :: 1895
mask prediction :: 1896
mask prediction :: 1897
mask prediction :: 1898
mask prediction :: 1899
mask prediction :: 1900
mask prediction :: 1901
mask prediction :: 1902
mask prediction :: 1903
mask prediction :: 1904
mask prediction :: 1905
mask prediction :: 1906
mask prediction :: 1907
mask prediction :: 1908
mask prediction :: 1909
mask prediction 

mask prediction :: 2229
mask prediction :: 2230
mask prediction :: 2231
mask prediction :: 2232
mask prediction :: 2233
mask prediction :: 2234
mask prediction :: 2235
mask prediction :: 2236
mask prediction :: 2237
mask prediction :: 2238
mask prediction :: 2239
mask prediction :: 2240
mask prediction :: 2241
mask prediction :: 2242
mask prediction :: 2243
mask prediction :: 2244
mask prediction :: 2245
mask prediction :: 2246
mask prediction :: 2247
mask prediction :: 2248
mask prediction :: 2249
mask prediction :: 2250
mask prediction :: 2251
mask prediction :: 2252
mask prediction :: 2253
mask prediction :: 2254
mask prediction :: 2255
mask prediction :: 2256
mask prediction :: 2257
mask prediction :: 2258
mask prediction :: 2259
mask prediction :: 2260
mask prediction :: 2261
mask prediction :: 2262
mask prediction :: 2263
mask prediction :: 2264
mask prediction :: 2265
mask prediction :: 2266
mask prediction :: 2270
mask prediction :: 2271
mask prediction :: 2272
mask prediction 

mask prediction :: 2589
mask prediction :: 2590
mask prediction :: 2591
mask prediction :: 2592
mask prediction :: 2593
mask prediction :: 2594
mask prediction :: 2595
mask prediction :: 2596
mask prediction :: 2597
mask prediction :: 2598
mask prediction :: 2599
mask prediction :: 2600
mask prediction :: 2601
mask prediction :: 2602
mask prediction :: 2603
mask prediction :: 2604
mask prediction :: 2605
mask prediction :: 2606
mask prediction :: 2607
mask prediction :: 2608
mask prediction :: 2609
mask prediction :: 2610
mask prediction :: 2611
mask prediction :: 2612
mask prediction :: 2613
mask prediction :: 2614
mask prediction :: 2615
mask prediction :: 2616
mask prediction :: 2617
mask prediction :: 2618
mask prediction :: 2619
mask prediction :: 2620
mask prediction :: 2621
mask prediction :: 2622
mask prediction :: 2623
mask prediction :: 2624
mask prediction :: 2625
mask prediction :: 2626
mask prediction :: 2627
mask prediction :: 2628
mask prediction :: 2629
mask prediction 

(0.7741819347905855, 0.6315634895632284)

In [116]:
mask_pred_2d = raw_img_class.reconstruct_original_from_padded_tiled(mask_pred_rle)

In [145]:
mask_pred.shape, mask_label_2d.shape

((18484, 13013), (18484, 13013))

In [155]:
mask_pred_2d = raw_img_class.reconstruct_original_from_padded_tiled(mask_pred_rle)

masks_train = '/media/bony/Ganga_HDD_3TB/Ganges_Backup/Machine_Learning/HuBMAP_Hacking_Kidney/hubmap-kidney-segmentation/train.csv'
mask_rle_df = pd.read_csv(masks_train).set_index('id')
mask_rle_df.head()
img_file_name = img_file.split('.')[0]
mask_rle = mask_rle_df.loc[img_file_name, 'encoding']
dx = raw_img_class.dx
dy = raw_img_class.dy

mask_label_2d = mask_rle_to_2d(mask_rle, dx, dy)
metric_dice_iou(mask_pred_2d, mask_label_2d, smooth = 0.005)

59457.0 4854.0 4256919.0
64311.0 4316376


(0.027145058495225683, 0.013759278013714522)